In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

import numpy as np
import matplotlib.pyplot as plt
import SolveLinSys
from utilities import compute_derivatives_3d

Equation:

$$
0 = \max_{e} \min_{h_2} b\left[\delta \eta \log e - \delta(1-\eta)\lambda_1 y z_2 + \xi_m\frac{(h_2)^2}{2}\right] -\ell e + \frac{\partial \phi}{\partial y}(y, b, z_2;\ell) e + \left[\frac{\partial^2 \phi}{\partial y^2}(y,b, z_2;\ell)\right] \epsilon - \frac{\partial \phi}{\partial b}(y, b, z_2;\ell) \delta b + \left[\frac{\partial \phi}{\partial z_2}(y, b, z_2;\ell)\right]\left[-\rho (z_2 -\mu_2) + \sqrt{z_2} \sigma_2 h_2\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(y,b, z_2;\ell)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$

FOC of $h_2$ gives : 

$$
h_2^* = -\frac{\left[\frac{\partial \phi}{\partial z_2}(y, b, z_2;\ell)\right]\sqrt{z_2}\sigma_2}{b\xi_m}
$$

FOC of $e$ gives :

$$
e^* = \frac{b \delta \eta}{\ell - \frac{\partial \phi}{\partial y}(y, b, z_2; \ell)}
$$

In [2]:
def false_transient_one_iteration(stateSpace, A, B_z, B_b, B_y, C_zz, C_bb, C_yy, D, v0, ε=.3, tol=-10):
    A = A.reshape(-1, 1, order='F')
    B = np.hstack([B_z.reshape(-1, 1, order='F'), B_b.reshape(-1, 1, order='F'), B_y.reshape(-1, 1, order='F')])
    C = np.hstack([C_zz.reshape(-1, 1, order='F'), C_bb.reshape(-1, 1, order='F'), C_yy.reshape(-1, 1, order='F')])
    D = D.reshape(-1, 1, order='F')
    v0 = v0.reshape(-1, 1, order='F')
    out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε, tol)
    return out


def false_transient(z_grid, b_grid, y_grid, ℓ, model_paras=(), v0=None, ϵ=.5, tol=1e-8, max_iter=10_000):
    τ, η, ξ_m, δ, η, μ_2, ρ, σ_2 = model_paras
    γ_2 = 0.

    Δ_z = z_grid[1] - z_grid[0]
    Δ_b = b_grid[1] - b_grid[0]
    Δ_y = y_grid[1] - y_grid[0]
    (z_mat, b_mat, y_mat) = np.meshgrid(z_grid, b_grid, y_grid, indexing = 'ij')
    stateSpace = np.hstack([z_mat.reshape(-1, 1, order='F'), b_mat.reshape(-1, 1, order='F'), y_mat.reshape(-1, 1, order='F')])

    if v0 is None:
        v0 = np.zeros_like(z_mat)

    count = 1
    error = 1.
    
    while error > tol and count < max_iter:
        v_old = v0.copy()
        v0_dz = compute_derivatives_3d(v0, 0, 1, Δ_z)
        v0_dzz = compute_derivatives_3d(v0, 0, 2, Δ_z)
        v0_db = compute_derivatives_3d(v0, 1, 1, Δ_b)
        v0_dy = compute_derivatives_3d(v0, 2, 1, Δ_y)

        e = b_mat*δ*η / (ℓ-v0_dy)
        e[e<=0] = 1e-10
        h = -v0_dz*np.sqrt(z_mat)*σ_2 / (b_mat*ξ_m)
        
        A = np.zeros_like(z_mat)
        B_z = -ρ*(z_mat - μ_2) + np.sqrt(z_mat)*σ_2*h
        B_b = -(δ*b_mat)
        B_y = e
        C_zz = z_mat*σ_2**2/2
        C_bb = np.zeros_like(z_mat)
        C_yy = np.zeros_like(z_mat)
        D = b_mat*(δ*η*np.log(e) - 1.7675/10000*δ*(1-η)*y_mat*z_mat - γ_2*z_mat**2*y_mat**2/2 + ξ_m*h**2/2) - ℓ*e

        res = false_transient_one_iteration(stateSpace, A, B_z, B_b, B_y, C_zz, C_bb, C_yy, D, v0, ϵ) # CG tol = 1e-10
        v0 = res[2].reshape(v0.shape, order = "F")

        rhs_error = A*v0 + B_z*v0_dz + B_b*v0_db + B_y*v0_dy + C_zz*v0_dzz + D
        rhs_error = np.max(abs(rhs_error))
        lhs_error = np.max(abs((v0 - v_old)/ϵ))
        error = lhs_error
        
        print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))
        count += 1
    print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))
    return v0, e

In [4]:
# Model parameters
γ = .018
τ = 1.86/1000 * γ
η = .032
ξ_m = .00256
δ = .01
η = .032

μ_2 = 1.86/1000.
ρ = 0.9
σ_2 = np.sqrt(2*ρ/10416.29)/2

model_paras = (τ, η, ξ_m, δ, η, μ_2, ρ, σ_2)

# Grid setting
n_z = 20
z_min = np.max([μ_2 - 6*np.sqrt(σ_2**2*μ_2/(2*ρ)), 1e-9])
z_max = μ_2 + 6*np.sqrt(σ_2**2*μ_2/(2*ρ))
z_grid = np.linspace(z_min, z_max, n_z)

n_b = 20
b_min = 1e-1
b_max = 1.
b_grid = np.linspace(b_min, b_max, n_b)

n_y = 20
y_min = 1e-2
y_max = 3000.
y_grid = np.linspace(y_min, y_max, n_y)

# ℓ setting
log_ell_grid = np.linspace(-13, -5, 100)
ℓ_step = 1e-7

In [6]:
import time
start = time.time()
ψ, e = false_transient(z_grid, b_grid, y_grid, 1e-7, model_paras=model_paras, v0=None, ϵ=.5, tol=1e-8, max_iter=10_000)
print(time.time()-start)

Iteration 1: LHS Error: 0.022919501723948927; RHS Error 0.015362031812286094
Iteration 2: LHS Error: 0.009613475920821086; RHS Error 0.010824401791615153
Iteration 3: LHS Error: 0.008778261890674995; RHS Error 0.009612536822724792
Iteration 4: LHS Error: 0.00823088980301441; RHS Error 0.008797476812793147
Iteration 5: LHS Error: 0.007850826976817604; RHS Error 0.008242826912454594
Iteration 6: LHS Error: 0.007580059015060092; RHS Error 0.00785695923446006
Iteration 7: LHS Error: 0.007383386896350583; RHS Error 0.007582785131454383
Iteration 8: LHS Error: 0.007238050582268812; RHS Error 0.007384267818220353
Iteration 9: LHS Error: 0.00712881820813871; RHS Error 0.007237985909530477
Iteration 10: LHS Error: 0.007045249500725256; RHS Error 0.00712830468861848
Iteration 11: LHS Error: 0.00698005666306642; RHS Error 0.007044553303170414
Iteration 12: LHS Error: 0.006928077153744244; RHS Error 0.006979315307136018
Iteration 13: LHS Error: 0.0068856106489645075; RHS Error 0.0069273577528474
I

Iteration 121: LHS Error: 0.004423351431119271; RHS Error 0.004443411889779219
Iteration 122: LHS Error: 0.004403362041486725; RHS Error 0.004423345027749351
Iteration 123: LHS Error: 0.0043834502510176065; RHS Error 0.0044033557813345885
Iteration 124: LHS Error: 0.004363616030332862; RHS Error 0.0043834441300156495
Iteration 125: LHS Error: 0.004343859341729761; RHS Error 0.004363610044563148
Iteration 126: LHS Error: 0.004324180139505085; RHS Error 0.004343853487400746
Iteration 127: LHS Error: 0.004304578370290302; RHS Error 0.00432417441297249
Iteration 128: LHS Error: 0.004285053973329789; RHS Error 0.004304572768025994
Iteration 129: LHS Error: 0.0042656068714866935; RHS Error 0.004285048491932287
Iteration 130: LHS Error: 0.004246237012496823; RHS Error 0.004265601534441781
Iteration 131: LHS Error: 0.004226944299540736; RHS Error 0.004246231789734243
Iteration 132: LHS Error: 0.004207728646237863; RHS Error 0.004226939187995737
Iteration 133: LHS Error: 0.004188589959363553; R

Iteration 233: LHS Error: 0.002633007285233857; RHS Error 0.002645406080120762
Iteration 234: LHS Error: 0.002620663664250733; RHS Error 0.0026330063998131806
Iteration 235: LHS Error: 0.002608375890279291; RHS Error 0.0026206627916810048
Iteration 236: LHS Error: 0.0025961437380288555; RHS Error 0.0026083750303472703
Iteration 237: LHS Error: 0.0025839669824407885; RHS Error 0.002596142890517101
Iteration 238: LHS Error: 0.002571845398866568; RHS Error 0.00258396614713637
Iteration 239: LHS Error: 0.0025597787631852498; RHS Error 0.0025718445755772744
Iteration 240: LHS Error: 0.0025477668518316676; RHS Error 0.0025597779517047455
Iteration 241: LHS Error: 0.002535809441801762; RHS Error 0.0025477660519632283
Iteration 242: LHS Error: 0.002523906310650803; RHS Error 0.0025358086533609
Iteration 243: LHS Error: 0.0025120572364643046; RHS Error 0.002523905533449091
Iteration 244: LHS Error: 0.002500261997863351; RHS Error 0.002512056470310293
Iteration 245: LHS Error: 0.0024885203740039

Iteration 344: LHS Error: 0.001557254915551587; RHS Error 0.0015646793209645976
Iteration 345: LHS Error: 0.001549865008101614; RHS Error 0.0015572547194948327
Iteration 346: LHS Error: 0.00154250964074798; RHS Error 0.0015498648144741422
Iteration 347: LHS Error: 0.0015351886578496288; RHS Error 0.00154250944951651
Iteration 348: LHS Error: 0.0015279019044660558; RHS Error 0.0015351884689875073
Iteration 349: LHS Error: 0.0015206492263093452; RHS Error 0.0015279017179484825
Iteration 350: LHS Error: 0.0015134304697266288; RHS Error 0.0015206490420990395
Iteration 351: LHS Error: 0.0015062454816876514; RHS Error 0.0015134302877871392
Iteration 352: LHS Error: 0.0014990941097925425; RHS Error 0.0015062453019955568
Iteration 353: LHS Error: 0.0014919762022467253; RHS Error 0.0014990939323103478
Iteration 354: LHS Error: 0.0014848916078806784; RHS Error 0.0014919760269526748
Iteration 355: LHS Error: 0.0014778401761388338; RHS Error 0.0014848914347453074
Iteration 356: LHS Error: 0.001470

Iteration 457: LHS Error: 0.0009081552944607552; RHS Error 0.0009125108415129926
Iteration 458: LHS Error: 0.0009038202674978013; RHS Error 0.0009081552645234385
Iteration 459: LHS Error: 0.000899505766757569; RHS Error 0.0009038202380425549
Iteration 460: LHS Error: 0.0008952117089044975; RHS Error 0.0008995057377865647
Iteration 461: LHS Error: 0.0008909379996495659; RHS Error 0.0008952116803918023
Iteration 462: LHS Error: 0.0008866845428117109; RHS Error 0.0008909379716055799
Iteration 463: LHS Error: 0.0008824512425935627; RHS Error 0.0008866845152131959
Iteration 464: LHS Error: 0.0008782380041159055; RHS Error 0.0008824512154569624
Iteration 465: LHS Error: 0.0008740447333481782; RHS Error 0.0008782379774287619
Iteration 466: LHS Error: 0.0008698713369654776; RHS Error 0.0008740447071069585
Iteration 467: LHS Error: 0.0008657177221949031; RHS Error 0.0008698713111400336
Iteration 468: LHS Error: 0.0008615837967638207; RHS Error 0.0008657176967979971
Iteration 469: LHS Error: 0.0

...

Iteration 9423: LHS Error: 4.7665837126301125e-08; RHS Error 0.00017515595561396564
Iteration 9424: LHS Error: 4.766532950978286e-08; RHS Error 0.00017515594421195968
Iteration 9425: LHS Error: 4.7665664826192916e-08; RHS Error 0.00017515595577871558
Iteration 9426: LHS Error: 4.766515723786391e-08; RHS Error 0.00017515594437571215
Iteration 9427: LHS Error: 4.766549255600869e-08; RHS Error 0.000175155955945103
Iteration 9428: LHS Error: 4.766498500757832e-08; RHS Error 0.0001751559445405381
Iteration 9429: LHS Error: 4.766532030317169e-08; RHS Error 0.00017515595610780586
Iteration 9430: LHS Error: 4.766481279117052e-08; RHS Error 0.0001751559447033497
Iteration 9431: LHS Error: 4.7665148088498616e-08; RHS Error 0.00017515595627047662
Iteration 9432: LHS Error: 4.766464058170161e-08; RHS Error 0.00017515594486829198
Iteration 9433: LHS Error: 4.7664975887703326e-08; RHS Error 0.0001751559564361169
Iteration 9434: LHS Error: 4.7664468441621644e-08; RHS Error 0.00017515594503283104
Iter

Iteration 9543: LHS Error: 4.765395722299004e-08; RHS Error 0.00017515595474010964
Iteration 9544: LHS Error: 4.7651766393874695e-08; RHS Error 0.00017515596613491705
Iteration 9545: LHS Error: 4.765571301251423e-08; RHS Error 0.00017515596829374713
Iteration 9546: LHS Error: 4.765367853879626e-08; RHS Error 0.0001751559550103828
Iteration 9547: LHS Error: 4.765146829205369e-08; RHS Error 0.00017515596640013643
Iteration 9548: LHS Error: 4.765543821973889e-08; RHS Error 0.0001751559685608398
Iteration 9549: LHS Error: 4.76534005797169e-08; RHS Error 0.0001751559552753191
Iteration 9550: LHS Error: 4.7651180237751056e-08; RHS Error 0.0001751559666614543
Iteration 9551: LHS Error: 4.765516417202728e-08; RHS Error 0.00017515596882067795
Iteration 9552: LHS Error: 4.765312331105748e-08; RHS Error 0.00017515595553067073
Iteration 9553: LHS Error: 4.7650899760720566e-08; RHS Error 0.00017515596691857556
Iteration 9554: LHS Error: 4.7654890813000894e-08; RHS Error 0.00017515596907494158
Itera

Iteration 9663: LHS Error: 4.764352906638791e-08; RHS Error 0.0001751559646152184
Iteration 9664: LHS Error: 4.7641491376926304e-08; RHS Error 0.00017515597599528283
Iteration 9665: LHS Error: 4.764509378869797e-08; RHS Error 0.0001751559781481021
Iteration 9666: LHS Error: 4.764327941886304e-08; RHS Error 0.00017515596485938745
Iteration 9667: LHS Error: 4.764124177103479e-08; RHS Error 0.00017515597623689837
Iteration 9668: LHS Error: 4.76448347923146e-08; RHS Error 0.00017515597839419116
Iteration 9669: LHS Error: 4.764302979909374e-08; RHS Error 0.00017515596510156656
Iteration 9670: LHS Error: 4.7640992227593326e-08; RHS Error 0.00017515597648536032
Iteration 9671: LHS Error: 4.7644576002363326e-08; RHS Error 0.00017515597863366084
Iteration 9672: LHS Error: 4.764278031810232e-08; RHS Error 0.00017515596534767655
Iteration 9673: LHS Error: 4.76407427535408e-08; RHS Error 0.00017515597672894672
Iteration 9674: LHS Error: 4.764431741624206e-08; RHS Error 0.00017515597888305976
Itera

Iteration 9783: LHS Error: 4.763356804826646e-08; RHS Error 0.00017515597434000752
Iteration 9784: LHS Error: 4.763153089309968e-08; RHS Error 0.00017515598571624308
Iteration 9785: LHS Error: 4.763485774014406e-08; RHS Error 0.00017515598787157737
Iteration 9786: LHS Error: 4.7633318719930706e-08; RHS Error 0.00017515597458424238
Iteration 9787: LHS Error: 4.7631281613336185e-08; RHS Error 0.00017515598596467674
Iteration 9788: LHS Error: 4.763460431612618e-08; RHS Error 0.00017515598811604133
Iteration 9789: LHS Error: 4.7633069329144906e-08; RHS Error 0.00017515597483103921
Iteration 9790: LHS Error: 4.763103219479481e-08; RHS Error 0.00017515598620639517
Iteration 9791: LHS Error: 4.763435098925281e-08; RHS Error 0.00017515598835810258
Iteration 9792: LHS Error: 4.76328198342757e-08; RHS Error 0.00017515597507416515
Iteration 9793: LHS Error: 4.763078268604781e-08; RHS Error 0.0001751559864506257
Iteration 9794: LHS Error: 4.7634097753018745e-08; RHS Error 0.0001751559886036913
Ite

Iteration 9904: LHS Error: 4.761840183536287e-08; RHS Error 0.00017515599919188567
Iteration 9905: LHS Error: 4.7618312762516546e-08; RHS Error 0.00017515599928361416
Iteration 9906: LHS Error: 4.761822361594448e-08; RHS Error 0.00017515599937338014
Iteration 9907: LHS Error: 4.7618134406922363e-08; RHS Error 0.00017515599946188703
Iteration 9908: LHS Error: 4.7618045125909225e-08; RHS Error 0.00017515599955182518
Iteration 9909: LHS Error: 4.761795578504813e-08; RHS Error 0.00017515599963992007
Iteration 9910: LHS Error: 4.7617866396482134e-08; RHS Error 0.00017515599972588787
Iteration 9911: LHS Error: 4.76177769385272e-08; RHS Error 0.00017515599981336304
Iteration 9912: LHS Error: 4.7617687429397926e-08; RHS Error 0.0001751559999015862
Iteration 9913: LHS Error: 4.761759787343112e-08; RHS Error 0.00017515599998610897
Iteration 9914: LHS Error: 4.76175082723615e-08; RHS Error 0.00017515600007061147
Iteration 9915: LHS Error: 4.761741861317864e-08; RHS Error 0.0001751560001574911
Ite

In [ ]:
# fig = go.Figure(data=[go.Surface(z=ψ[10,:,:])])

# fig.update_layout(title='Mt Bruno Elevation', autosize=False,
#                   width=500, height=500,
#                   margin=dict(l=65, r=50, b=65, t=90))

# fig.show()

plt.figure()
plt.plot(y_grid, e[10,9,:])
plt.title(r'Figure 1: $e(y,b,z_2)$ when $z_2=0.00186$, $b=1.$, $\ell=10^{-7}$ ')
plt.xlabel('y')
plt.ylabel('e')
plt.savefig('fig_1.pdf')

plt.figure()
plt.plot(y_grid, ψ[10,9,:])
plt.title(r'Figure 2: $\phi(y,b,z_2)$ when $z_2=0.00186$, $b=1$, $\ell=10^{-7}$ ')
plt.xlabel('y')
plt.ylabel('ϕ')
plt.savefig('fig_2.pdf')

In [ ]:
ψs = []
ψs_next = []
ψ_guess = np.load('phi_guess.npy')

for i, ℓ in enumerate(np.exp(log_ell_grid)):
    print(i)
    max_iter = 1000
    ψ = false_transient(z_grid, b_grid, y_grid, ℓ, model_paras=model_paras, v0=ψ_guess, ϵ=.5, tol=1e-6, max_iter=max_iter)
    ψ_next = false_transient(z_grid, b_grid, y_grid, ℓ+ℓ_step, model_paras=model_paras, v0=ψ_guess, ϵ=.5, tol=1e-6, max_iter=max_iter)
    ψs.append(ψ)
    ψs_next.append(ψ_next)
    ψ_guess = ψ.copy()

In [ ]:
ϕs = []
rs = []
distortions = []
ψs_new = []
ℓs = []
for i in range(len(ψs)):
    ψ_b1_i = ψs[i][:, -1, :]
    ψ_b1_ip1 = ψs_next[i][:, -1, :]
    dψ = (ψ_b1_ip1-ψ_b1_i)/ℓ_step
    ψ = (ψ_b1_ip1+ψ_b1_i)/2.
    ℓ = np.exp(log_ell_grid)[i] + ℓ_step/2.
    r = -dψ
    ϕ = ψ + ℓ*r
    ψs_new.append(ψ)
    ϕs.append(ϕ)
    rs.append(r)
    Δ_z = z_grid[1] - z_grid[0]
    dϕdz = np.zeros_like(ψ)
    for j in range(n_z):
        if j == 0:
            dϕdz[j] = (ψ[j+1, :]-ψ[j, :])/Δ_z
        elif j == n_z-1:
            dϕdz[j] = (ψ[j, :]-ψ[j-1, :])/Δ_z
        else:
            dϕdz[j] = (ψ[j+1, :]-ψ[j-1, :])/(2*Δ_z)
    distortion = -(dϕdz*z_grid*σ_2**2)/ξ_m
    distortions.append(distortion)
    ℓs.append(ℓ)

In [ ]:
z_loc = 10
y_loc = 19

distortion_z = np.array([dist[z_loc, y_loc] for dist in distortions])
r_z = np.array([r[z_loc, y_loc] for r in rs])
ϕ_z = np.array([ϕ])
ℓs = np.array(ℓs)
plt.figure()
plt.title('Figure 3: distortion when $z_2=0.00186$, $y=3000$')
plt.xlabel(r'$\ell$')
plt.plot(ℓs, distortion_z)
plt.savefig('fig_3.pdf')
# plt.figure()
# plt.plot(r_z, distortion_z)
# plt.xlim(0, 2500)
# plt.plot(ℓs, r_z)

In [ ]:
# ψ = false_transient(z_grid, b_grid, y_grid, 1e-7, model_paras=model_paras, v0=None, ϵ=.5, tol=1e-8, max_iter=4000)

In [ ]:
np.save('phi_guess.npy', ψ)

In [ ]:
# # fig = go.Figure(data=[go.Surface(z=ψ[10,:,:])])

# # fig.update_layout(title='Mt Bruno Elevation', autosize=False,
# #                   width=500, height=500,
# #                   margin=dict(l=65, r=50, b=65, t=90))

# # fig.show()

# plt.figure()
# plt.plot(y_grid, e[10,9,:])
# plt.title(r'Figure 1: $\phi(y,b,z_2)$ when $z_2=1.86/1000$, $b=0.5$ ')
# plt.xlabel('y')
# plt.ylabel('e')
# # plt.savefig('fig_1.pdf')

# plt.figure()
# plt.plot(y_grid, ψ[10,9,:])
# plt.title(r'Figure 2: $\phi(y,b,z_2)$ when $z_2=1.$, $b=0.01$ ')
# plt.xlabel('y')
# plt.ylabel('ϕ')
# # plt.savefig('fig_2.pdf')

In [ ]:
# plt.figure()
# plt.plot(y_grid, ψ[10,20,:])
# plt.title(r'Figure 1: $\phi(y,b,z_2)$ when $z_2=1.$, $b=0.5$ ')
# plt.xlabel('y')
# plt.ylabel('ϕ')
# plt.savefig('fig_3.pdf')

In [ ]:
# from numba import njit
# @njit
# def simulate_z(μ, ρ, σ, T):
#     Ws = np.random.normal(0., 1., T)
#     Zs = np.zeros(T)
#     Zs[0] = μ
#     for i in range(1, T):
#         Zs[i] = Zs[i-1] - ρ*(Zs[i-1]-μ) + np.sqrt(Zs[i-1])*σ*Ws[i]
#     return Zs

In [ ]:
# res = simulate_z(μ_2, .99, 0.004, 1_000_000)
# plt.hist(res, bins=2000, label=r'Var(Z) = .004')
# plt.title(r'Figure 1: $\mu_2 = 1$, $\rho = .99$, $\sigma_2 = .004$')
# plt.legend()
# plt.savefig('fig_1.pdf')
# plt.show()

In [ ]:
# np.std(res)*5

In [ ]:
# np.percentile(res, 1)

In [ ]:
# res = simulate_z(μ_2, 1e-8, 2.97e-5, 100_000_000)

In [ ]:
# plt.hist(res, bins=2000, label=r'Var(Z) = 0.21')
# plt.title(r'Figure 2: $\mu_2 = 1$, $\rho = 10^{-8}$, $\sigma_2 = 3*10^{-5}$')
# plt.legend()
# plt.savefig('fig_2.pdf')
# plt.show()